In [1]:
import sys; sys.path.append("../")
import re
import datetime
import io

import pandas as pd

from clean.baton_rouge_fpcsb_lprr import clean
from lib.rows import duplicate_row
from lib.explore import show_unique, print_df
from datamatch import (
    ColumnsIndex, JaroWinklerSimilarity, StringSimilarity, DateSimilarity, ThresholdMatcher
)

In [2]:
df = clean()

df

../clean/baton_rouge_fpcsb_lprr.py:87: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, "docket_no"] = df.docket_no.fillna("").str.strip()\
../clean/baton_rouge_fpcsb_lprr.py:103: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, "hearing_date"] = df.hearing_date.fillna("").str.strip()\
../clean/baton_rouge_fpcsb_lprr.py:176: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, "hearing_date"] = df.hearing_date.str.replace(r"\/\/", "/")\
../clean/baton_rouge_fpcsb_lprr.py:194: FutureWarning: The default value of regex will change from True to False in a future version.
  names = df.name.str.strip().str.replace("Tat Chi-Lam", "Tat-Chi Lam")\
../clean/baton_rouge_fpcsb_lprr.py:199: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, "counsel"] = names.loc[:, 1].str.lower(

,docket_no,action,resolution,hearing_year,hearing_month,hearing_day,counsel,rank_desc,last_name,first_name,middle_name,middle_initial,agency,data_production_year,uid,appeal_uid,resolution_uid
0,10-03,20-day suspension,overturned by board (01-19-2012),2012,1,15,dirks,,nacoste,otis,,,Baton Rouge PD,2012,8537a0cbc5c961c3b462335f16c30937,955d379040041b70160acdb8bf801de3,a530be4c671fbc74497fc986e52250a8
1,11-10,termination,"board approved continuance to may 17, 2012 per...",2012,2,16,crystal lafleur,,moore,alicia,,m,Baton Rouge PD,2012,85004c499ffd7fd49a12b98fae3595cc,6e621709202b9d3f76d45fe0d5bf1b27,ca6de1775bcf497a4f1098fd04a8426b
2,11-11,15-day suspension,rescheduled,2012,3,15,dirk,,schilling,robert,,,Baton Rouge PD,2012,f05a482450ff7ea0f29029be17870d0b,eb2c4d96036dace64cf8b7db1ea3abe6,118188e1b977834b8c405e02d8e49936
3,12-03,termination,board upheld chief white's decision to terminate,2012,4,19,,,hollis,pamela,,b,Baton Rouge PD,2012,2d2accf099c55a18be22321a3c45bda6,dcdf0bb3a37a50655d6a2cba08455b35,09a9d4e8cfd1421b80ae425aa7f08b3e
4,11-10,termination,appealed to 19th judicial court,2012,5,17,crystal lafleur,,moore,alicia,,m,Baton Rouge PD,2012,85004c499ffd7fd49a12b98fae3595cc,6e621709202b9d3f76d45fe0d5bf1b27,c06fff01910f9feeb81fc26ab36babd0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,06-05,1-day suspension,board upheld chief le duff's decision,2007,10,18,falcon,,batiste,nicholas,,k,Baton Rouge PD,2012,3d540b4c365931f90024a5c76c45453d,e478fe5faefd7ebad12590b35c4f0643,d76fe9976716393125d1d69aacc377a4
374,07-05,1-day suspension,board upheld,2007,10,18,dirks,,landry iii,carroll,,j,Baton Rouge PD,2012,d98e8d9f5fb3bf14c6cbb1622666827a,fd873982feb8bff63d9dd045f4b356e9,0a3133658b4fc117cde5cda138eace95
375,07-06,termination,stayed criminal investigation,2007,12,16,dirks,,davis,nathan,,,Baton Rouge PD,2012,a796d4519c78d2c9eb898fdd7c98d929,c88ddefc54e09b9a100b755ff87d500c,7c21c62989c77b966c43e80c4052607f
376,07-06,termination,NaN,2007,12,20,dirks iii,,davis,nathan,,,Baton Rouge PD,2012,a796d4519c78d2c9eb898fdd7c98d929,c88ddefc54e09b9a100b755ff87d500c,da66cfc7c9531cbd489253b4d332b8f2


In [8]:
df = df.drop_duplicates()
df[df.duplicated(subset=['resolution_uid'])]

,docket_no,action,resolution,hearing_year,hearing_month,hearing_day,counsel,rank_desc,last_name,first_name,middle_name,middle_initial,agency,data_production_year,uid,appeal_uid,resolution_uid


In [4]:
pprr17 = pd.read_csv('../data/match/pprr_baton_rouge_csd_2017.csv')

dfa = df[['uid', 'first_name', 'last_name', 'middle_initial']]
dfa.loc[:, 'fc'] = dfa.first_name.map(lambda x: x[:1])
dfa = dfa.drop_duplicates(subset=['uid']).set_index('uid')

dfb = pprr17[['uid', 'first_name', 'last_name', 'middle_initial']]
dfb.loc[:, 'fc'] = dfb.first_name.map(lambda x: x[:1])
dfb = dfb.drop_duplicates(subset=['uid']).set_index('uid')

matcher = ThresholdMatcher(dfa, dfb, ColumnsIndex(["fc"]), {
    "first_name": JaroWinklerSimilarity(),
    "last_name": JaroWinklerSimilarity(),
    "middle_initial": StringSimilarity(),
})
print_df(matcher.get_sample_pairs(sample_counts=5, step=0.025))

/Users/khoipham/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/khoipham/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1675: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
                                                                    first_name     last_name middle_initial fc
score_range pair_idx sim_score row_key                        

In [5]:
pprr19 = pd.read_csv('../data/match/pprr_baton_rouge_csd_2019.csv')

dfa = df[['uid', 'first_name', 'last_name', 'middle_initial']]
dfa.loc[:, 'fc'] = dfa.first_name.map(lambda x: x[:1])
dfa = dfa.drop_duplicates(subset=['uid']).set_index('uid')

dfb = pprr19[['uid', 'first_name', 'last_name', 'middle_initial']]
dfb.loc[:, 'fc'] = dfb.first_name.map(lambda x: x[:1])
dfb = dfb.drop_duplicates(subset=['uid']).set_index('uid')

matcher = ThresholdMatcher(dfa, dfb, ColumnsIndex(["fc"]), {
    "first_name": JaroWinklerSimilarity(),
    "last_name": JaroWinklerSimilarity(),
    "middle_initial": StringSimilarity(),
})
print_df(matcher.get_sample_pairs(sample_counts=5, step=0.025))

/Users/khoipham/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/khoipham/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1675: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
                                                                first_name   last_name middle_initial fc
score_range pair_idx sim_score row_key                              

In [6]:
post = pd.read_csv('../data/clean/pprr_post_2020_11_06.csv')
post = post[post.agency == 'baton rouge pd']

dfa = df[['uid', 'first_name', 'last_name']]
dfa.loc[:, 'fc'] = dfa.first_name.map(lambda x: x[:1])
dfa = dfa.drop_duplicates(subset=['uid']).set_index('uid')

dfb = post[['uid', 'first_name', 'last_name']]
dfb.loc[:, 'fc'] = dfb.first_name.map(lambda x: x[:1])
dfb = dfb.drop_duplicates(subset=['uid']).set_index('uid')

matcher = ThresholdMatcher(dfa, dfb, ColumnsIndex(["fc"]), {
    "first_name": JaroWinklerSimilarity(),
    "last_name": JaroWinklerSimilarity(),
})
print_df(matcher.get_sample_pairs(sample_counts=10, step=0.025))

/Users/khoipham/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/khoipham/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1675: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
                                                                  first_name     last_name fc
score_range pair_idx sim_score row_key                                         

In [7]:
len(matcher.get_index_pairs_within_thresholds(0.93))

86